In [1]:
# 全局设置
import os
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

# HDB = QS.FactorDB.HDF5DB(sys_args={"主目录": "../Data/HDF5"}).connect()
HDB = QS.FactorDB.HDF5DB(config_file="../config/HDF5DBConfig.json").connect()

# RDB = QS.RiskDB.HDF5FRDB(sys_args={"主目录":"../Data/RiskData"}).connect();
RDB = QS.RiskDB.HDF5FRDB(config_file="../config/HDF5FRDBConfig.json").connect();

# 均值方差模型

均值方差模型来源于上世纪 50 年代的 Markowiz 均值方差理论, 即在给定约束条件下期望效用最大化. 其导出的优化问题的一般形式如下：

$$
\begin{aligned}
  & \underset{\mathbf{w}}{\mathop{\max }}\,\left\{\gamma\cdot\mathbf{\mu}^T\cdot\mathbf{w}-\frac{\lambda }{2}\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}-\operatorname{TC}\left( \mathbf{w} \right) \right\} \\ 
 & \operatorname{TC}\left( \mathbf{w} \right)={\lambda_1}\sum\limits_{i=1}^n{\left| {{w}_{i}}-{{w}_{0i}} \right|} + {\lambda_2}\sum\limits_{i=1}^n{\left( {{w}_{i}}-{{w}_{0i}} \right)^{+}} + {\lambda_3}\sum\limits_{i=1}^n{{\left( {{w}_{i}}-{{w}_{0i}} \right)}^{-}} \\
 & s.t.\ \mathbf{w}\in\mathfrak{C}
\end{aligned}
$$
其中, $\mathbf{w}$ 是组合权重, 优化变量, $\mathbf{\mu}$ 是预期收益率, $\mathbf{\Sigma}$ 是收益率的预期协方差矩阵, $\gamma$ 是收益项系数, $\lambda\ge 0$ 是风险厌恶系数, $\operatorname{TC}$ 是交易成本惩罚函数, $\lambda_1\ge 0$ 是买卖交易费率, $\lambda_2\ge 0$ 是买入交易费率, $\lambda_3\ge 0$ 是卖出交易费率, $\mathbf{w}_0$ 是当前持有的组合权重, $\mathfrak{C}$ 是约束条件自由组合成的约束集. 

均值方差模型导出的优化问题通常是一个凸规划问题(如果含有非零权重数目约束条件, 则为整数规划问题), 在某些参数取值特殊的情况下可以退化为二次规划或者线性规划问题, 很多成熟的凸优化算法都可以有效的求解该问题. 

如果不对权重做任何约束, 原始的均值方差模型有很多问题:
1. 参数估计误差大。Chopra 和 Ziemba（1993）的研究表明，在风险厌恶为 50 的情况下，均值估计误差带来的效用损失远远高于方差和协方差；风险厌恶水平越高，对均值的估计误差越敏感，效用损失越大。参数尤其是预期收益的估计误差，会给结果带来巨大的不确定性，带来 "Garbage In, Garbage Out" 的后果。
2. 结果对参数输入非常敏感。Michaud（1989）发现，其结果可能极其不稳定，输入参数的较小改变，可能会使结果大相径庭。
3. 优化结果可能过于集中。Broadie（1993）的测试表明，在约束条件欠缺的的时候，均值方差模型的结果容易集中在少数证券甚至一个证券。
4. 换手率高，交易成本太大。De Carvalho 、 Lu 和 Moulin（2012）比较了 6 个组合优化模型（市值加权、等权重、风险平价、波动率倒数、均值方差 和最大分散度），结果表明均值方差模型换手率较高，在不加卖空约束时换手率更高。
5. 容易得到极端的分配结果。Best 和 Grauer（1991）的研究表明，均值方差模型容易算出极大或极小的权重，且结果对输入均值异常敏感。
6. 较差的样本外表现。DeMiguel et al.（2009）的结果表明，基于历史数据的均值方差组合，由于估计误差的存在，在样本外表现很难超越等权重组合。

Behr、Guettler 和 Miebs（2013），从权重约束的角度对最小方差组合进行了改进。由于最小方差模型持仓过于集中，换手率高，Behr et al.（2013）尝试进行改进，一方面试图超越等权组合，另一方面减少换手率。具体来说，通过最小化协方差矩阵的 MSE 得到上限和下限，即将样本协方差向一个特定的协方差压缩，这个特定的协方差由上限和下限决定。在获得权重上限和下限后，再带入最小方差组合求解最小方差解即可。利用 5 个数据集，评估权重约束最小方差组合的表现。结果发现：权重约束最小方差组合夏普比等权重组合高 30%，比市值加权高 60%，比简单的卖空约束组合和单因子协方差组合也实现了更高的夏普；和 Demiguel et al.（2009）提出的 PMV 相比，同样是少数跑赢等权组合的方案，权重约束最小方差组合换手率更低。


In [2]:
# 组合优化
PC = QS.PortfolioConstructor.CVXPC()

# 设置相关数据
TargetDT = dt.datetime(2018, 7, 31)
FT = HDB.getTable("stock_cn_day_bar_nafilled")
TargetIDs = FT.getID()[:500]
PC.Args["目标ID"] = TargetIDs
PC.Args["预期收益"] = FT.readData(factor_names=["chg_rate"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
RT = RDB.getTable("BarraRiskData")
#PC["协方差矩阵"] = RT.readCov(dts=[TargetDT], ids=TargetIDs)[TargetDT]
PC.Args["因子协方差阵"] = RT.readFactorCov(dts=[TargetDT]).loc[TargetDT]
PC.Args["风险因子"] = RT.readFactorData(dts=[TargetDT], ids=TargetIDs).loc[:, TargetDT, :]
PC.Args["特异性风险"] = RT.readSpecificRisk(dts=[TargetDT], ids=TargetIDs).loc[TargetDT, :]
PC.Args["成交金额"] = FT.readData(factor_names=["amount"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
PC.Args["初始投资组合"] = pd.Series(0.0,index=TargetIDs)
PC.Args["总财富"] = 1000000000

# 设置优化目标
Objective = QS.PortfolioConstructor.MeanVarianceObjective(pc=PC)
Objective.Args["收益项系数"] = 0.0
Objective.Args["风险厌恶系数"] = 1.0
PC.Args["优化目标"] = Objective

# 设置约束条件
# 预算约束
iConstraint = QS.PortfolioConstructor.BudgetConstraint(pc=PC)
iConstraint.Args["限制上限"] = 1.0
iConstraint.Args["限制下限"] = 1.0
PC.Args["约束条件"].append(iConstraint)
# 权重约束
iConstraint = QS.PortfolioConstructor.WeightConstraint(pc=PC)
iConstraint.Args["限制上限"] = 1.0
iConstraint.Args["限制下限"] = 0.0
PC.Args["约束条件"].append(iConstraint)

# 求解优化问题
Portfolio, ResultInfo = PC.solve()

print(Portfolio)
print(ResultInfo)

000001.SZ    0.000005
000002.SZ    0.000006
000004.SZ    0.000005
000005.SZ    0.000009
000006.SZ    0.000009
               ...   
000985.SZ    0.000011
000987.SZ    0.000008
000988.SZ    0.000006
000989.SZ    0.000002
000990.SZ    0.006829
Length: 450, dtype: float64
{'Status': 1, 'Msg': 'optimal', 'solver_name': 'OSQP', 'solve_time': 0.42990800704184856, 'setup_time': None, 'num_iters': 275, 'ReleasedConstraint': []}
